In [ ]:
# 📘 Title: Visualization of Antibiotic Resistance Gene (ARG) Annotations
# 📍 Description: Plotting ARG summary using barplots, heatmaps, and pie charts

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# === Step 1: Load the Data ===
data_dir = Path("../results/ARG")
summary_file = data_dir / "ARG_summary_table.csv"
annotated_file = data_dir / "ARG_annotations_combined.csv"

summary_df = pd.read_csv(summary_file)
annotated_df = pd.read_csv(annotated_file)

# === Step 2: Bar Plot - Top 10 Most Abundant ARGs ===
top_n = 10
top_args = summary_df.nlargest(top_n, "count")

plt.figure(figsize=(10, 6))
sns.barplot(data=top_args, x="count", y="gene_name", palette="magma")
plt.xlabel("Count")
plt.ylabel("ARG Gene")
plt.title("Top 10 Most Abundant ARGs")
plt.tight_layout()
plt.savefig(data_dir / "figures/ARG_barplot_top10.png", dpi=300)
plt.show()

# === Step 3: Heatmap - ARG vs Sample Matrix ===
pivot_df = annotated_df.groupby(["sample", "gene_name"]).size().reset_index(name="count")
heatmap_df = pivot_df.pivot(index="gene_name", columns="sample", values="count").fillna(0)

plt.figure(figsize=(10, max(6, 0.3 * len(heatmap_df))))
sns.heatmap(heatmap_df, cmap="YlOrBr", linewidths=0.5, annot=True, fmt=".0f")
plt.title("ARG Abundance Heatmap")
plt.tight_layout()
plt.savefig(data_dir / "figures/ARG_heatmap.png", dpi=300)
plt.show()

# === Step 4: Pie Chart - ARG Distribution ===
total = summary_df["count"].sum()
summary_df["percentage"] = summary_df["count"] / total * 100
top_pie = summary_df.nlargest(8, "count")  # top 8 genes
others = pd.DataFrame({
    "gene_name": ["Others"],
    "count": [summary_df["count"].sum() - top_pie["count"].sum()],
    "percentage": [100 - top_pie["percentage"].sum()]
})
pie_df = pd.concat([top_pie, others])

plt.figure(figsize=(8, 8))
plt.pie(pie_df["percentage"], labels=pie_df["gene_name"], autopct="%1.1f%%", startangle=140, colors=sns.color_palette("pastel"))
plt.title("ARG Gene Distribution")
plt.tight_layout()
plt.savefig(data_dir / "figures/ARG_piechart.png", dpi=300)
plt.show()
